## Prepare Prompt data for fine tuning


In [ ]:
!pip install datasets


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'validation':'validation.csv', 'test': 'test.csv'})
dataset

In [ ]:
# Shuffle the dataset and slice it
dataset['train'] = dataset['train'].shuffle(seed=42)

# Define a function to transform the data
def transform_conversation(example):
    prompt = 'If you are a licensed psychologist, please provide this client with a helpful response to her concern.'
    formatted_prompt = f"<s>[INST] <<SYS>> {prompt} <</SYS>> {example['Question']} [/INST] {example['Answer']} </s>"

    return {'Prompt': formatted_prompt}


# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)

In [ ]:
dataset['train'][0]
transformed_dataset['train'][0]

In [ ]:
transformed_dataset.shape

In [ ]:
!pip install \
    torch==2.0.1 --quiet \
    transformers==4.31.0 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    accelerate==0.21.0 \
    loralib==0.1.1 \
    peft==0.4.0 \
    bitsandbytes==0.40.2 \
    trl==0.4.7

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model name
new_model = "llama-2-7b-therapy"
peft_model_path = './drive/MyDrive/aitherapy/peft'
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./drive/MyDrive/aitherapy/results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 500

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load tokenizer and model with QLoRA configuration
check_point_path = "./drive/MyDrive/aitherapy/results/Llama-2-7b-chat-hf/"
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset['train'],
    peft_config=peft_config,
    dataset_text_field="Prompt",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
# trainer.train()

# Save trained model
# trainer.model.save_pretrained(new_model)

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
del model
del tokenizer
del trainer
del dataset
torch.cuda.empty_cache()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
fine_tuned_model = PeftModel.from_pretrained(
    model,
    check_point_path,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
    is_trainable=False
)
#model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Run text generation pipeline with our next model
prompt = 'If you are a licensed psychologist, please provide this client with a helpful response to her concern.'
user_prompt = "I feel stressed at work"
formatted_prompt = f"<s>[INST] <<SYS>> {prompt} <</SYS>> {user_prompt} [/INST]"

pipe = pipeline(task="text-generation", model=fine_tuned_model, tokenizer=tokenizer, max_length=200)
result = pipe(formatted_prompt)
print(result[0]['generated_text'])

In [ ]:
print(result[0]['generated_text'])

In [ ]:
!pip install gradio
import gradio as gr

In [ ]:
def format_chat_prompt(message, chat_history):
    sys_prompt = "Suppose you are a licensed psychologist, please provide this client with a helpful response to her concern.\n"
    prompts = []
    prompt = ''
    prompts.append(f"<s>[INST] <<SYS>> {sys_prompt} <</SYS>>")
    for turn in chat_history:
        user_message, bot_message = turn
        prompts.append(f" {user_message} [/INST] {bot_message} </s>")
    prompts.append(f"{prompt}\n<s>[INST] <<SYS>> {sys_prompt} <</SYS>> {message} [/INST]")
    prompt = "\n".join(prompts)
    return prompt

def respond(message, chat_history):
        formatted_prompt = format_chat_prompt(message, chat_history)
        pipe = pipeline(task="text-generation", model=fine_tuned_model, tokenizer=tokenizer, max_length=200)
        result = pipe(formatted_prompt)
        print(result[0]['generated_text'])
        bot_message = result[0]['generated_text']

        chat_history.append((message, bot_message))
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit
gr.close_all()
demo.launch(debug=True, share=True)